RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [2]:
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: functions_coding ninjas.pdf


Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)


  ✓ Loaded 9 pages

Total documents loaded: 9


In [4]:
all_pdf_documents

[Document(metadata={'producer': 'Mac OS X 10.10.5 Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20170618173632Z00'00'", 'title': 'Microsoft Word -  C1_L5_Functions.docx', 'moddate': "D:20170618173632Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '..\\data\\pdf\\functions_coding ninjas.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1', 'source_file': 'functions_coding ninjas.pdf', 'file_type': 'pdf'}, page_content='C++\t\r \xa0Foundation\t\r \xa0with\t\r \xa0Data\t\r \xa0Structures\t\r \xa0Lecture\t\r \xa05\t\r \xa0\t\r \xa0:\t\r \xa0Functions,\t\r \xa0Variables\t\r \xa0and\t\r \xa0Scope\t\r \xa0s'),
 Document(metadata={'producer': 'Mac OS X 10.10.5 Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20170618173632Z00'00'", 'title': 'Microsoft Word -  C1_L5_Functions.docx', 'moddate': "D:20170618173632Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '..\\data\\pdf\\functions_coding ninjas.pdf', 'total_pages': 9, 'page': 1, 'page_label': '2', 's

In [5]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [6]:
chunks=split_documents(all_pdf_documents)
chunks

Split 9 documents into 25 chunks

Example chunk:
  s...e	les		
Metadata: {'producer': 'Mac OS X 10.10.5 Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20170618173632Z00'00'", 'title': 'Microsoft Word -  C1_L5_Functions.docx', 'moddate': "D:20170618173632Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '..\\data\\pdf\\functions_coding ninjas.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1', 'source_file': 'functions_coding ninjas.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Mac OS X 10.10.5 Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20170618173632Z00'00'", 'title': 'Microsoft Word -  C1_L5_Functions.docx', 'moddate': "D:20170618173632Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '..\\data\\pdf\\functions_coding ninjas.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1', 'source_file': 'functions_coding ninjas.pdf', 'file_type': 'pdf'}, page_content='C++\t\r \xa0Foundation\t\r \xa0with\t\r \xa0Data\t\r \xa0Structures\t\r \xa0Lecture\t\r \xa05\t\r \xa0\t\r \xa0:\t\r \xa0Functions,\t\r \xa0Variables\t\r \xa0and\t\r \xa0Scope\t\r \xa0s'),
 Document(metadata={'producer': 'Mac OS X 10.10.5 Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20170618173632Z00'00'", 'title': 'Microsoft Word -  C1_L5_Functions.docx', 'moddate': "D:20170618173632Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '..\\data\\pdf\\functions_coding ninjas.pdf', 'total_pages': 9, 'page': 1, 'page_label': '2', 's

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity
import os

In [5]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


c:\Users\farma\Desktop\AI-ML\YTRAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\farma\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

Model loaded successfully. Embedding dimension: 384


VectorStore

In [9]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0
